## Finding Differences Between Summaries by Topic Using GPT-4
This notebook uses the GPT-4 model to compare and highlight the differences between two summaries. The primary steps involve:<br><br>
* Configuration and Setup<br>
* Loading Document Summaries<br>
* Comparing and Highlighting Differences<br>

#### 1. Configuration and Setup
1.1. Import Necessary Libraries

In [1]:
# Importing necessary libraries for chat modeling, prompts, logging, etc.
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import logging
import yaml
import os

1.2. Logging Setup

In [2]:
# Set up logging to capture progress and potential issues
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

1.3. Load API Key and Set Paths

In [3]:
# Load the API key from the credentials file
with open('./../credentials/oai-key.yml', 'rb') as f:
    credentials = yaml.safe_load(f)

api_key = credentials['key']
os.environ['OPENAI_API_KEY'] = api_key

# Define paths for input and output data
LOCAL_INPUT_DIR = './DATA/INPUT'
LOCAL_OUTPUT_DIR = './DATA/OUTPUT'

# Specify the model name
MODEL_NAME = 'gpt-4'

# Define file names for the summaries to be compared
FILE_NAME_1 = 'file-1'
FILE_NAME_2 = 'file-2'

#### 2. Loading Document Summaries

In [4]:
# Load the summaries of the two documents for comparison
with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME_1}/SUMMARY/{FILE_NAME_1}-summary-vai.txt', 'rb') as f:
    summary_1 = f.read()

with open(f'{LOCAL_OUTPUT_DIR}/{FILE_NAME_2}/SUMMARY/{FILE_NAME_2}-summary-vai.txt', 'rb') as f:
    summary_2 = f.read()

#### 3. Comparing and Highlighting Differences
3.1. Initialize the GPT-4 Model

In [5]:
# Initialize the GPT-4 model for chat-based tasks
llm = ChatOpenAI(model_name=MODEL_NAME, 
                 temperature=0.0, 
                 max_tokens=2048)

3.2. Define Chat Prompts

In [6]:
# Define the system prompt for introducing the task to the model
system_template = """
You are a Derivatives Risk Analyst tasked with comparing the summaries of two documents, SUMMARY_1 and SUMMARY_2 provided below.
---
SUMMARY_1 => {summary_1}
---
SUMMARY_2 => {summary_2}
---
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Define the human prompt to instruct the model on how to approach the comparison
human_template = """
First, identify common themes between them. 
Next, point out the differences between the identified topics.
Finally, write a detailed paragraph on the identified differences.
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Combine the system and human prompts
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
prompt = chat_prompt.format_prompt(summary_1=summary_1, summary_2=summary_2).to_messages()

3.3. Execute the Model and Retrieve Response

In [7]:
# Execute the model with the formulated prompt and log the response
response = llm(prompt)
logger.info(response.content)

Common Themes:
1. Calculation of Adjusted Notional Amount: Both summaries discuss the calculation of the adjusted notional amount for different types of derivative contracts such as equity, commodity, interest rate, credit, and exchange rate derivative contracts.
2. Maturity Factor: Both summaries provide details on how to calculate the maturity factor of a derivative contract that is not subject to a variation margin agreement.
3. Supervisory Delta Adjustment: Both summaries discuss the supervisory delta adjustment for derivative contracts, particularly for option contracts.

Differences:
1. Haircuts for Market Price Volatility: SUMMARY_1 provides detailed information on the haircuts for market price volatility based on different residual maturities and exposure types. This information is not present in SUMMARY_2.
2. Minimum Period of Risk (MPOR): SUMMARY_1 discusses the MPOR for a derivative contract based on the type of contract and the counterparty. This topic is not covered in SUM